In [53]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
#from millify import millify

Read the File from Data folder and remove all duplicates. 

In [54]:
p = Path().cwd()
print(f"original File path: {p}")
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
print(f"Updated File Path to pick from Data Library: { stringpath}")
ngsimfile=stringpath + '/' + 'Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data.csv'
ngsim = pd.read_csv(ngsimfile,low_memory=False)
print(f"{ngsim.duplicated().sum()} duplicate values have been removed")
ngsim.drop_duplicates(inplace=True)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Updated File Path to pick from Data Library: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data
704000 duplicate values have been removed


Feet to Metres:

Below Four parametrs consider distance in Feet and they are converted to metre. Speed to Metre/ second from Feet/second and acceleration from feet/second Square to Metre/Second Square. 

Drop the columns we wont use in the model.
Add Relative time which is the time from the first timframe in the dataset. This is calculated by using the minimum value ofGlobal time(absolute time from 1970)

In [55]:
ngsim['v_length']= ngsim['v_length']*0.3048
ngsim['Space_Headway']= ngsim['Space_Headway']*0.3048
ngsim['v_Vel']= ngsim['v_Vel']*0.3048
ngsim['v_Acc']= ngsim['v_Acc']*0.3048
ngsim['Relative_Time']= ngsim['Global_Time'] - ngsim['Global_Time'].min() + 1
ngsim= ngsim.drop(columns=['Movement', 'Direction','Section_ID','Int_ID','D_Zone','O_Zone','Following','v_Width','Total_Frames'])


Create Placeholders for the user defined columns we will be populating later, to be used in the model. 
Map the Vehicle classes to their english equivalent. 
Filter the original file into US-101 and I-80 to perform and populate the required fields later. These files contain same values of Vehicle ID used for multiple vehicles based on the highway. 



In [56]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim['Preceding_Vehicle_Class'] = np.NaN
ngsim['Rear_to_Front_Space_Headway'] = np.NaN
ngsim['Front_To_Rear_Time_Headway'] = np.NaN
ngsim['L-F_Pair'] = ngsim['Preceding'].astype(str) + '-' + ngsim['Vehicle_ID'].astype(str)
ngsim['Velocity Difference_Following-Preceding'] = np.NaN
ngsim['Acceleration Difference_Following-Preceding'] = np.NaN

ngsim["v_Class_Name"] = ngsim["v_Class"].map({1:"Motorcycle", 2: "Car", 3: "Heavy Vehicle"})
filtered_ngsim_U= ngsim[((ngsim['Location'] == 'us-101')) ]
#These two vehicles have a bad trajectory and keep on following each other in a single lane in a matter of 
filtered_ngsim_I= ngsim[(ngsim['Location'] == 'i-80') & ( ((ngsim['Vehicle_ID'] != 1748 )  & (ngsim['Preceding'] != 1734 )) &(  (ngsim['Vehicle_ID'] != 1734 )  & (ngsim['Preceding'] != 1748 )))]


Need to dig deeper into this as Preceding is greater than 0 but the space between the Lead and Follower have no space. 

In [57]:
filtered_ngsim_U[((filtered_ngsim_U['Space_Headway'] == 0 ) & (filtered_ngsim_U['Preceding'] > 0 )) ]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Front_To_Rear_Time_Headway,L-F_Pair,Velocity Difference_Following-Preceding,Acceleration Difference_Following-Preceding,v_Class_Name
76361,99,1065,1118847948500,12.942,1420.624,6452132.084,1872437.889,2.5908,1,17.373600,...,0.0,us-101,1117684929401,NaN,NaN,NaN,92-99,NaN,NaN,Motorcycle
107058,806,3878,1118848229800,6.699,796.216,6451666.005,1872853.511,4.4196,2,2.795016,...,0.0,us-101,1117685210701,NaN,NaN,NaN,814-806,NaN,NaN,Car
452347,366,2149,1118848971900,12.613,835.132,6451695.487,1872819.732,2.1336,1,8.695944,...,0.0,us-101,1117685952801,NaN,NaN,NaN,370-366,NaN,NaN,Motorcycle
3595333,2871,8411,1118847820000,7.631,547.070,6451508.896,1872991.251,3.9624,2,2.944368,...,0.0,us-101,1117684800901,NaN,NaN,NaN,2876-2871,NaN,NaN,Car
5900431,2964,8731,1118847852000,4.868,309.729,6451336.355,1873154.258,5.0292,2,2.618232,...,0.0,us-101,1117684832901,NaN,NaN,NaN,2956-2964,NaN,NaN,Car


In [58]:
filtered_ngsim_I[((filtered_ngsim_I['Space_Headway'] == 0 ) & (filtered_ngsim_I['Preceding'] > 0 )) ]

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,Time_Headway,Location,Relative_Time,Preceding_Vehicle_Class,Rear_to_Front_Space_Headway,Front_To_Rear_Time_Headway,L-F_Pair,Velocity Difference_Following-Preceding,Acceleration Difference_Following-Preceding,v_Class_Name
5910808,2506,7263,1113437493200,53.343,380.297,6042837.478,2133451.781,4.35864,2,3.514344,...,0.00,i-80,1112274474101,NaN,NaN,NaN,2512-2506,NaN,NaN,Car
6759942,2491,7916,1113437558500,17.161,598.884,6042774.601,2133664.163,4.66344,2,3.453384,...,0.00,i-80,1112274539401,NaN,NaN,NaN,2486-2491,NaN,NaN,Car
9365776,572,3292,1113437894100,17.928,1485.487,6042644.407,2134541.664,4.35864,2,0.000000,...,9999.99,i-80,1112274875001,NaN,NaN,NaN,566-572,NaN,NaN,Car
9803270,1394,4030,1113437169900,76.566,444.910,6042852.523,2133518.460,4.51104,2,1.865376,...,0.00,i-80,1112274150801,NaN,NaN,NaN,1407-1394,NaN,NaN,Car


Create Sets to use and create the L-F Pairs. 

In [59]:
filtered_vClass_U= filtered_ngsim_U[['Vehicle_ID','v_Class','v_length']].drop_duplicates().sort_values('v_Class')
v_Class_M_U     =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 1 )  ]['Vehicle_ID'])
v_Class_C_U     =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 2 )  ]['Vehicle_ID'])
v_Class_HV_U    =set(filtered_vClass_U[(filtered_vClass_U['v_Class'] == 3 )  ]['Vehicle_ID'])


filtered_vClass_I= filtered_ngsim_I[['Vehicle_ID','v_Class','v_length']].drop_duplicates().sort_values('v_Class')
v_Class_M_I     =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 1 )  ]['Vehicle_ID'])
v_Class_C_I     =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 2 )  ]['Vehicle_ID'])
v_Class_HV_I    =set(filtered_vClass_I[(filtered_vClass_I['v_Class'] == 3 )  ]['Vehicle_ID'])

Add the Preceding Vehicle Class to the new previous class column

In [60]:
result = []

for i in filtered_ngsim_U.index:
    if filtered_ngsim_U['Preceding'][i] in v_Class_C_U:
        result.append('Car')
    elif filtered_ngsim_U['Preceding'][i] in v_Class_HV_U:
        result.append('Heavy Vehicle')
    elif filtered_ngsim_U['Preceding'][i] in v_Class_M_U:
        result.append('Motorcycle')
    else: 
        result.append('Free Flow')
filtered_ngsim_U['Preceding_Vehicle_Class'] = result

result = []

for i in filtered_ngsim_I.index:
    if filtered_ngsim_I['Preceding'][i] in v_Class_C_I:
        result.append('Car')
    elif filtered_ngsim_I['Preceding'][i] in v_Class_HV_I:
        result.append('Heavy Vehicle')
    elif filtered_ngsim_I['Preceding'][i] in v_Class_M_I:
        result.append('Motorcycle')
    else: 
        result.append('Free Flow')
filtered_ngsim_I['Preceding_Vehicle_Class'] = result

Map the Vehicle Length of the previous vehicle into the row. 

In [61]:
vehicle_lengths_U= filtered_ngsim_U[['Vehicle_ID','v_length']]
print(f"{vehicle_lengths_U.duplicated().sum()} duplicate values have been removed")
vehicle_lengths_U.drop_duplicates(inplace=True)
print(vehicle_lengths_U.shape)
x_U=vehicle_lengths_U.groupby(['Vehicle_ID']).mean()
dict_U=x_U.to_dict()['v_length']
filtered_ngsim_U["previous_Vehicle_length"] = filtered_ngsim_U["Preceding"].map(dict_U)

4093130 duplicate values have been removed
(5803, 2)


In [62]:
vehicle_lengths_I= filtered_ngsim_I[['Vehicle_ID','v_length']]
print(f"{vehicle_lengths_I.duplicated().sum()} duplicate values have been removed")
vehicle_lengths_I.drop_duplicates(inplace=True)
print(vehicle_lengths_I.shape)
x_I=vehicle_lengths_I.groupby(['Vehicle_ID']).mean()
dict_I=x_I.to_dict()['v_length']
filtered_ngsim_I["previous_Vehicle_length"] = filtered_ngsim_I["Preceding"].map(dict_I)

4550407 duplicate values have been removed
(5532, 2)


Convert the Front to Front Space Headway to Back Bumper (preceding Vehicle) to front Bumper(of Following ) Headway. This will give the actual distance between the vehicle ignoring the vehicle length.

In [63]:
filtered_ngsim_U['Rear_to_Front_Space_Headway']=filtered_ngsim_U['Space_Headway'] - filtered_ngsim_U['previous_Vehicle_length']
filtered_ngsim_I['Rear_to_Front_Space_Headway']=filtered_ngsim_I['Space_Headway'] - filtered_ngsim_I['previous_Vehicle_length']

Find the time it would take for the Following vehicle to reach the rear Bumper of the Preceding Vehicle. 

In [64]:
filtered_ngsim_U['Front_To_Rear_Time_Headway'] = filtered_ngsim_U['Rear_to_Front_Space_Headway']  / filtered_ngsim_U['v_Vel'] 
filtered_ngsim_I['Front_To_Rear_Time_Headway'] = filtered_ngsim_I['Rear_to_Front_Space_Headway']  / filtered_ngsim_I['v_Vel'] 

Vehicle Combination of the Preceding and the Following Vehicle. 

In [65]:
filtered_ngsim_U['Vehicle_combination']= filtered_ngsim_U['Preceding_Vehicle_Class'] + '-' + filtered_ngsim_U['v_Class_Name']
filtered_ngsim_I['Vehicle_combination']= filtered_ngsim_I['Preceding_Vehicle_Class'] + '-' + filtered_ngsim_I['v_Class_Name']


1. Velocity difference of the Lead minus Preceding Vehicle.
2. Acceleration difference of the Lead minus Preceding Vehicle.
3. Find the Pair details of the Lead_following 
4. Find the cumulative time the Pair runs within the highway as a pair. 

In [66]:
right_filtered_ngsim_U=filtered_ngsim_U[['Preceding','Relative_Time','v_Vel','v_Acc']]
right_filtered_ngsim_U.rename(columns={'Preceding':'Prec_Vehicle_ID','v_Vel':'previous_Vehicle_Velocity','v_Acc':'previous_Vehicle_Acceleration'}, inplace=True)
filtered_ngsim_U['Prec_Vehicle_ID']=filtered_ngsim_U['Vehicle_ID']
filtered_ngsim_U=filtered_ngsim_U.merge(right=right_filtered_ngsim_U,how='left', on=('Prec_Vehicle_ID','Relative_Time'))
filtered_ngsim_U['Velocity Difference_Following-Preceding'] = filtered_ngsim_U['v_Vel'] - filtered_ngsim_U['previous_Vehicle_Velocity']
filtered_ngsim_U['Acceleration Difference_Following-Preceding'] = filtered_ngsim_U['v_Acc'] - filtered_ngsim_U['previous_Vehicle_Acceleration']
filtered_ngsim_U=filtered_ngsim_U.sort_values(by=['Relative_Time'],ascending=True,ignore_index=True)
filtered_ngsim_U['Pair_Time_Duration']=(filtered_ngsim_U.groupby(['L-F_Pair'],as_index=False).cumcount()*0.1) +0.1

In [67]:
right_filtered_ngsim_I=filtered_ngsim_I[['Preceding','Relative_Time','v_Vel','v_Acc']]
right_filtered_ngsim_I.rename(columns={'Preceding':'Prec_Vehicle_ID','v_Vel':'previous_Vehicle_Velocity','v_Acc':'previous_Vehicle_Acceleration'}, inplace=True)
filtered_ngsim_I['Prec_Vehicle_ID']=filtered_ngsim_I['Vehicle_ID']
filtered_ngsim_I=filtered_ngsim_I.merge(right=right_filtered_ngsim_I,how='left', on=('Prec_Vehicle_ID','Relative_Time'))
filtered_ngsim_I['Velocity Difference_Following-Preceding'] = filtered_ngsim_I['v_Vel'] - filtered_ngsim_I['previous_Vehicle_Velocity']
filtered_ngsim_I['Acceleration Difference_Following-Preceding'] = filtered_ngsim_I['v_Acc'] - filtered_ngsim_I['previous_Vehicle_Acceleration']
filtered_ngsim_I=filtered_ngsim_I.sort_values(by=['Relative_Time'],ascending=True,ignore_index=True)
filtered_ngsim_I['Pair_Time_Duration']=(filtered_ngsim_I.groupby(['L-F_Pair'],as_index=False).cumcount()*0.1) +0.1

In [68]:
filtered_ngsim= pd.concat([filtered_ngsim_U , filtered_ngsim_I])
print(filtered_ngsim_U.shape[0])
print(filtered_ngsim_I.shape[0])
print(filtered_ngsim.shape[0])
stringpath= str(p)[0:str(p).rfind('\\')] + '\\data'
ngsimfilteredfile=stringpath + '\\' + 'Cleaned_NGSIM_Data.csv'
filtered_ngsim.to_csv(ngsimfilteredfile, index = False)

4098933
4555939
8654872


In [69]:
filtered_ngsim

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Class,v_Vel,...,L-F_Pair,Velocity Difference_Following-Preceding,Acceleration Difference_Following-Preceding,v_Class_Name,previous_Vehicle_length,Vehicle_combination,Prec_Vehicle_ID,previous_Vehicle_Velocity,previous_Vehicle_Acceleration,Pair_Time_Duration
0,5,8,1118846979700,39.788,39.154,6451122.815,1873326.569,5.18160,2,12.192000,...,0-5,NaN,NaN,Car,NaN,Free Flow-Car,5,NaN,NaN,0.1
1,5,9,1118846979800,39.767,43.153,6451125.503,1873323.608,5.18160,2,12.192000,...,0-5,NaN,NaN,Car,NaN,Free Flow-Car,5,NaN,NaN,0.2
2,5,10,1118846979900,39.747,47.154,6451128.192,1873320.646,5.18160,2,12.192000,...,0-5,NaN,NaN,Car,NaN,Free Flow-Car,5,NaN,NaN,0.3
3,5,11,1118846980000,39.726,51.154,6451130.881,1873317.684,5.18160,2,12.192000,...,0-5,NaN,NaN,Car,NaN,Free Flow-Car,5,NaN,NaN,0.4
4,5,12,1118846980100,39.705,55.153,6451133.569,1873314.723,5.18160,2,12.192000,...,0-5,NaN,NaN,Car,NaN,Free Flow-Car,5,NaN,NaN,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4555934,2485,11687,1113438733600,55.017,1635.221,6042657.148,2134695.350,4.20624,2,3.557016,...,0-2485,NaN,NaN,Car,NaN,Free Flow-Car,2485,NaN,NaN,5.9
4555935,2485,11688,1113438733700,55.024,1636.411,6042656.973,2134696.489,4.20624,2,3.307080,...,0-2485,NaN,NaN,Car,NaN,Free Flow-Car,2485,NaN,NaN,6.0
4555936,2485,11689,1113438733800,55.034,1637.675,6042656.773,2134697.792,4.20624,2,3.307080,...,0-2485,NaN,NaN,Car,NaN,Free Flow-Car,2485,NaN,NaN,6.1
4555937,2485,11690,1113438733900,55.037,1638.175,6042656.697,2134698.286,4.20624,2,3.307080,...,0-2485,NaN,NaN,Car,NaN,Free Flow-Car,2485,NaN,NaN,6.2


In [70]:
filtered_ngsim[((filtered_ngsim['Location'] == 'us-101')) & ((filtered_ngsim['Vehicle_ID'] <= 100 )  ) & ((filtered_ngsim['Preceding'] <= 100 )  )].to_csv('VERIFYFILE.CSV', index = False)